# Securing Federated Learning Final project

# Project: Federated Learning with Additive secret sharing

### Step 1: Create Data Owners

In [1]:
import syft as sy
import torch as th
hook = sy.TorchHook(th)
from torch import nn, optim

# create a couple workers

bob = sy.VirtualWorker(hook, id="bob")
alice = sy.VirtualWorker(hook, id="alice")

priest1 = sy.VirtualWorker(hook, id="priest1")
priest2 = sy.VirtualWorker(hook, id="priest2")
priest3 = sy.VirtualWorker(hook, id="priest3")

bob.add_workers([alice, priest1, priest2, priest3])
alice.add_workers([bob, priest1, priest2, priest3])

# A Toy Dataset
data = th.tensor([[0,0],[0,1],[1,0],[1,1.]], requires_grad=True)
target = th.tensor([[0],[0],[1],[1.]], requires_grad=True)

# get pointers to training data on each worker by
# sending some training data to bob and alice
bobs_data = data[0:2].send(bob)
bobs_target = target[0:2].send(bob)

alices_data = data[2:].send(alice)
alices_target = target[2:].send(alice)

### Step 2: Create Our Model

In [2]:
# Iniitalize A Toy Model
model = nn.Linear(2,1)

### Step 3: Send a Copy of The Model to Alice and Bob

In [3]:
bobs_model = model.copy().send(bob)
alices_model = model.copy().send(alice)

bobs_opt = optim.SGD(params=bobs_model.parameters(),lr=0.1)
alices_opt = optim.SGD(params=alices_model.parameters(),lr=0.1)

### Step 4: Train Bob's and Alice's Models (in parallel)

In [4]:
for i in range(10):

    # Train Bob's Model
    bobs_opt.zero_grad()
    bobs_pred = bobs_model(bobs_data)
    bobs_loss = ((bobs_pred - bobs_target)**2).sum()
    bobs_loss.backward()

    bobs_opt.step()
    bobs_loss = bobs_loss.get().data

    # Train Alice's Model
    alices_opt.zero_grad()
    alices_pred = alices_model(alices_data)
    alices_loss = ((alices_pred - alices_target)**2).sum()
    alices_loss.backward()

    alices_opt.step()
    alices_loss = alices_loss.get().data
    alices_loss

### Step 5: Share both updated Models for additive secret sharing

In [5]:
priest1.clear_objects()
priest2.clear_objects()
priest3.clear_objects()


<VirtualWorker id:priest3 #objects:0>

In [6]:
alices_weight_shared = alices_model.weight.data.fix_prec().share(priest1, priest2, priest3).get()
print(alices_weight_shared)
bobs_weight_shared = bobs_model.weight.data.fix_prec().share(priest1, priest2, priest3).get()
print(bobs_weight_shared)

alices_bias_shared = alices_model.bias.data.fix_prec().share(priest1, priest2, priest3).get()
print(alices_bias_shared)
bobs_bias_shared = bobs_model.bias.data.fix_prec().share(priest1, priest2, priest3).get()
print(bobs_bias_shared)



(Wrapper)>FixedPrecisionTensor>[AdditiveSharingTensor]
	-> (Wrapper)>[PointerTensor | me:73212489694 -> priest1:96789671556]
	-> (Wrapper)>[PointerTensor | me:31658234600 -> priest2:40764897487]
	-> (Wrapper)>[PointerTensor | me:29072830945 -> priest3:30028538639]
	*crypto provider: me*
(Wrapper)>FixedPrecisionTensor>[AdditiveSharingTensor]
	-> (Wrapper)>[PointerTensor | me:32662826884 -> priest1:59098941984]
	-> (Wrapper)>[PointerTensor | me:64334101404 -> priest2:17023136159]
	-> (Wrapper)>[PointerTensor | me:28821802065 -> priest3:85178814038]
	*crypto provider: me*
(Wrapper)>FixedPrecisionTensor>[AdditiveSharingTensor]
	-> (Wrapper)>[PointerTensor | me:27931524655 -> priest1:9270584623]
	-> (Wrapper)>[PointerTensor | me:47889344334 -> priest2:17056871942]
	-> (Wrapper)>[PointerTensor | me:20581613220 -> priest3:20752988731]
	*crypto provider: me*
(Wrapper)>FixedPrecisionTensor>[AdditiveSharingTensor]
	-> (Wrapper)>[PointerTensor | me:22228497855 -> priest1:31632467828]
	-> (Wrapper

In [7]:
print('p1: ', priest1._objects)
print('p2: ', priest2._objects)
print('p3: ', priest3._objects)

p1:  {96789671556: tensor([[3251199777952384777, 3513528718422891821]]), 59098941984: tensor([[1339530140321009906,  807749356701800276]]), 9270584623: tensor([3352514323871012274]), 31632467828: tensor([587092001470888419])}
p2:  {40764897487: tensor([[-3030296439814996947,   953794938863176128]]), 17023136159: tensor([[ 686910726850785587, 2609267915036542742]]), 17056871942: tensor([-1209193903088709804]), 63277764463: tensor([2678707559830787976])}
p3:  {30028538639: tensor([[ -220903338137386920, -4467323657286067796]]), 85178814038: tensor([[-2026440867171795235, -3417017271738342869]]), 20752988731: tensor([-2143320420782302468]), 26958784095: tensor([-3265799561301676487])}


In [8]:
print(alices_weight_shared)
print(bobs_weight_shared)



(Wrapper)>FixedPrecisionTensor>[AdditiveSharingTensor]
	-> (Wrapper)>[PointerTensor | me:73212489694 -> priest1:96789671556]
	-> (Wrapper)>[PointerTensor | me:31658234600 -> priest2:40764897487]
	-> (Wrapper)>[PointerTensor | me:29072830945 -> priest3:30028538639]
	*crypto provider: me*
(Wrapper)>FixedPrecisionTensor>[AdditiveSharingTensor]
	-> (Wrapper)>[PointerTensor | me:32662826884 -> priest1:59098941984]
	-> (Wrapper)>[PointerTensor | me:64334101404 -> priest2:17023136159]
	-> (Wrapper)>[PointerTensor | me:28821802065 -> priest3:85178814038]
	*crypto provider: me*


### Step 6: Average The Models

In [9]:
with th.no_grad():
    avg_weight = (alices_weight_shared + bobs_weight_shared).get().float_prec() / 2
    avg_bias = (alices_bias_shared + bobs_bias_shared).get().float_prec() / 2
    print(avg_weight)
    print(avg_bias)
    model.weight.set_(avg_weight)
    model.bias.set_(avg_bias)
    # model.weight.set_(avg_weight)
    # model.bias.set_(avg_bias)

tensor([[0.5840, 0.1510]])
tensor([-0.0450])


### Step 7: Rinse and Repeat

In [10]:
iterations = 10
worker_iters = 5

for a_iter in range(iterations):

    bobs_model = model.copy().send(bob)
    alices_model = model.copy().send(alice)

    bobs_opt = optim.SGD(params=bobs_model.parameters(), lr=0.1)
    alices_opt = optim.SGD(params=alices_model.parameters(), lr=0.1)

    for wi in range(worker_iters):
        # Train Bob's Model
        bobs_opt.zero_grad()
        bobs_pred = bobs_model(bobs_data)
        bobs_loss = ((bobs_pred - bobs_target) ** 2).sum()
        bobs_loss.backward()

        bobs_opt.step()
        bobs_loss = bobs_loss.get().data

        # Train Alice's Model
        alices_opt.zero_grad()
        alices_pred = alices_model(alices_data)
        alices_loss = ((alices_pred - alices_target) ** 2).sum()
        alices_loss.backward()

        alices_opt.step()
        alices_loss = alices_loss.get().data

    alices_weight_shared = alices_model.weight.data.fix_prec().share(priest1, priest2, priest3).get()
    bobs_weight_shared = bobs_model.weight.data.fix_prec().share(priest1, priest2, priest3).get()
    
    alices_bias_shared = alices_model.bias.data.fix_prec().share(priest1, priest2, priest3).get()
    bobs_bias_shared = bobs_model.bias.data.fix_prec().share(priest1, priest2, priest3).get()
    
    with th.no_grad():
        avg_weight = (alices_weight_shared + bobs_weight_shared).get().float_prec() / 2
        avg_bias = (alices_bias_shared + bobs_bias_shared).get().float_prec() / 2        
        model.weight.set_(avg_weight)
        model.bias.set_(avg_bias)
    
    print("Bob:" + str(bobs_loss) + " Alice:" + str(alices_loss))

Bob:tensor(0.0047) Alice:tensor(0.0143)
Bob:tensor(0.0013) Alice:tensor(0.0071)
Bob:tensor(0.0003) Alice:tensor(0.0033)
Bob:tensor(4.9044e-05) Alice:tensor(0.0015)
Bob:tensor(7.1462e-05) Alice:tensor(0.0007)
Bob:tensor(0.0001) Alice:tensor(0.0003)
Bob:tensor(0.0002) Alice:tensor(0.0002)
Bob:tensor(0.0002) Alice:tensor(8.5088e-05)
Bob:tensor(0.0002) Alice:tensor(4.3939e-05)
Bob:tensor(0.0001) Alice:tensor(2.6266e-05)


In [11]:
preds = model(data)
loss = ((preds - target) ** 2).sum()

In [12]:
print(preds)
print(target)
print(loss.data)

tensor([[0.0380],
        [0.0345],
        [0.9535],
        [0.9500]], grad_fn=<AddmmBackward>)
tensor([[0.],
        [0.],
        [1.],
        [1.]], requires_grad=True)
tensor(0.0073)
